In [1]:
# !pip install census
# !pip install us
import pandas as pd
from census import Census
# from us import states
import matplotlib.pyplot as plt
import numpy as np
from scipy import interpolate


In [2]:
'''
There is a way to do this using the Census API, but this approach was aborted


states_top10 = ['AL','FL','GA','LA','MS','NC','SC','TX','VA','PR']


fields = ['NAME','DP04_0001E','DP04_0001PE','DP04_0002E','DP04_0002PE',
          'DP04_0017E','DP04_0017PE','DP04_0018E','DP04_0018PE',
          'DP04_0019E','DP04_0019PE','DP04_0020E','DP04_0020PE',
          'DP04_0021E','DP04_0021PE','DP04_0022E','DP04_0022PE',
          'DP04_0023E','DP04_0023PE','DP04_0024E','DP04_0024PE',
          'DP04_0025E','DP04_0025PE']
c = Census('ded8139bdac52bb2f9890e356da2ce5edc05bb83')
df = c.acs5.state_county('DP04', states.FL.GA.LA.MS.NC.SC.TX.VA.PR.state_fips, Census.ALL)
df = c.acs5.state_county('DP04_0001E',states.FL.fips, '170')
df[0:5]
'''

"\nThere is a way to do this using the Census API, but this approach was aborted\n\n\nstates_top10 = ['AL','FL','GA','LA','MS','NC','SC','TX','VA','PR']\n\n\nfields = ['NAME','DP04_0001E','DP04_0001PE','DP04_0002E','DP04_0002PE',\n          'DP04_0017E','DP04_0017PE','DP04_0018E','DP04_0018PE',\n          'DP04_0019E','DP04_0019PE','DP04_0020E','DP04_0020PE',\n          'DP04_0021E','DP04_0021PE','DP04_0022E','DP04_0022PE',\n          'DP04_0023E','DP04_0023PE','DP04_0024E','DP04_0024PE',\n          'DP04_0025E','DP04_0025PE']\nc = Census('ded8139bdac52bb2f9890e356da2ce5edc05bb83')\ndf = c.acs5.state_county('DP04', states.FL.GA.LA.MS.NC.SC.TX.VA.PR.state_fips, Census.ALL)\ndf = c.acs5.state_county('DP04_0001E',states.FL.fips, '170')\ndf[0:5]\n"

In [3]:
# Identify the states and territories to include in the dataframe
states_top10 = ['Alabama','Florida','Georgia','Louisiana',
                'Mississippi','North Carolina','South Carolina',
                'Texas','Virginia','Puerto Rico']

states_string  = 'Alabama|Florida|Georgia|Louisiana|Mississippi|North Carolina|South Carolina|Texas|Virginia|Puerto Rico'

# Identify which fields from the DP04 Census Table to include in the dataframe
fields = ['GEO_ID','NAME','DP04_0001E','DP04_0001PE','DP04_0002E','DP04_0002PE',
          'DP04_0017E','DP04_0017PE','DP04_0018E','DP04_0018PE',
          'DP04_0019E','DP04_0019PE','DP04_0020E','DP04_0020PE',
          'DP04_0021E','DP04_0021PE','DP04_0022E','DP04_0022PE',
          'DP04_0023E','DP04_0023PE','DP04_0024E','DP04_0024PE',
          'DP04_0025E','DP04_0025PE', 'DP04_0026E', 'DP04_0026PE']

# Read in the appropriate CSV files download from the US Census Bureau
df_dec2000 = pd.read_csv(r'DECENNIALDPSLDS2000.DP4-2025-06-05T131049.csv')
#df_2010_1yr = pd.read_csv(r'ACSDP1Y2010.DP04-Data.csv')
df_2010 = pd.read_csv(r'ACSDP5Y2010.DP04-Data.csv')
df_2015 = pd.read_csv(r'ACSDP5Y2015.DP04-Data.csv')
df_2019 = pd.read_csv(r'ACSDP5Y2019.DP04-Data.csv')
df_2023 = pd.read_csv(r'ACSDP5Y2023.DP04-Data.csv')

# Retain only the desired fields
#df_2010_1yr = df_2010_1yr[fields]
#df_2010_1yr['Year'] = 2010
df_2010 = df_2010[fields]
df_2010['Year'] = 2009
df_2015 = df_2015[fields]
df_2015['Year'] = 2014
df_2019 = df_2019[fields]
df_2019['Year'] = 2018
df_2023 = df_2023[fields]
df_2023['Year'] = 2022

# Adjust dataframes to account for varying decade aggregations then join the files into a single dataframe
df_2015['DP04_0018E'][1:] = pd.to_numeric(df_2015['DP04_0018E'][1:], errors='ignore') + pd.to_numeric(df_2015['DP04_0017E'][1:], errors='ignore')
df_2015['DP04_0017E'][1:] = 0
df_2015['DP04_0018PE'][1:] = pd.to_numeric(df_2015['DP04_0018PE'][1:], errors='ignore') + pd.to_numeric(df_2015['DP04_0017PE'][1:], errors='ignore')
df_2015['DP04_0017PE'][1:] = 0

df_2019['DP04_0018E'][1:] = pd.to_numeric(df_2019['DP04_0018E'][1:], errors='ignore') + pd.to_numeric(df_2019['DP04_0017E'][1:], errors='ignore')
df_2019['DP04_0017E'][1:] = 0
df_2019['DP04_0018PE'][1:] = pd.to_numeric(df_2019['DP04_0018PE'][1:], errors='ignore') + pd.to_numeric(df_2019['DP04_0017PE'][1:], errors='ignore')
df_2019['DP04_0017PE'][1:] = 0

df_2010['DP04_0026PE'] = df_2010['DP04_0025PE']
df_2010['DP04_0026E'] = df_2010['DP04_0025E']
df_2010['DP04_0025PE'] = df_2010['DP04_0024PE']
df_2010['DP04_0025E'] = df_2010['DP04_0024E']
df_2010['DP04_0024PE'] = df_2010['DP04_0023PE']
df_2010['DP04_0024E'] = df_2010['DP04_0023E']
df_2010['DP04_0023PE'] = df_2010['DP04_0022PE']
df_2010['DP04_0023E'] = df_2010['DP04_0022E']
df_2010['DP04_0022PE'] = df_2010['DP04_0021PE']
df_2010['DP04_0022E'] = df_2010['DP04_0021E']
df_2010['DP04_0021PE'] = df_2010['DP04_0020PE']
df_2010['DP04_0021E'] = df_2010['DP04_0020E']
df_2010['DP04_0020PE'] = df_2010['DP04_0019PE']
df_2010['DP04_0020E'] = df_2010['DP04_0019E']
df_2010['DP04_0019E'][1:] = pd.to_numeric(df_2010['DP04_0018E'][1:], errors='ignore') + pd.to_numeric(df_2010['DP04_0017E'][1:], errors='ignore')
df_2010['DP04_0017E'][1:] = 0
df_2010['DP04_0018E'][1:] = 0
df_2010['DP04_0019PE'][1:] = pd.to_numeric(df_2010['DP04_0018PE'][1:], errors='ignore') + pd.to_numeric(df_2010['DP04_0017PE'][1:], errors='ignore')
df_2010['DP04_0017PE'][1:] = 0
df_2010['DP04_0018PE'][1:] = 0


###
### Create 2000 estimates at the county level
###

#df_2000 = df_2010_1yr.copy()
df_2000 = df_2010.copy()
state_totals = df_dec2000.iloc[[0],[1,3,5,7,9,11,13,15,17,19]]
state_FIPS = '01|12|13|22|28|37|45|48|51|72'
df_2000['Year'] = 2000

# Get state total from 2010 to find county percentage estimates
FIPS_state_dict_2000 = dict(zip(['01','12','13','22','28','37','45','48','51','72'],
                           [int(vol.replace(',','')) for vol in state_totals.iloc[0]]))


#FIPS_state_dict_2010 = dict(zip(['01','12','13','22','28','37','45','48','51','72'],
#                           [int(vol.replace(',','')) for vol in df_2010_1yr['DP04_0001PE'][1:11]]))
FIPS_state_dict_2010 = dict(zip(['01','12','13','22','28','37','45','48','51','72'],
                           [int(vol.replace(',','')) for vol in df_2010['DP04_0001PE'][1:11]]))


df_2000['DP04_0001E'][1:11] = [int(vol.replace(',','')) for vol in state_totals.iloc[0]]
df_2000.drop(index=df_2000.index[0], axis=0, inplace=True)
df_2000['state_total_2000'] = np.nan
df_2000['state_total_2010'] = np.nan

df_2000['GEO_ID_short'] = [id.split('US')[1][0:2] for id in df_2000['GEO_ID']]
#df_2000 = df_2000[df_2000['GEO_ID_short'].str.contains(state_FIPS)] # PROBLEM IS HERE

# Filter out unnecessary states and counties
for key in FIPS_state_dict_2010:
  df_2000.loc[df_2000['GEO_ID_short'] == key, 'state_total_2000'] = FIPS_state_dict_2000[key]
  df_2000.loc[df_2000['GEO_ID_short'] == key, 'state_total_2010'] = FIPS_state_dict_2010[key]

df_2000['tot_housing_perc'] = df_2000['DP04_0001PE'].astype(int)/df_2000['state_total_2010']
df_2000['occ_housing_perc'] = df_2000['DP04_0002E'].astype(int)/df_2000['state_total_2010']


# Populate 2000 dataframe with estimated counts
df_2000['DP04_0001E'] = (df_2000['state_total_2000'] * df_2000['tot_housing_perc'])
df_2000['DP04_0001PE'] = df_2000['DP04_0001E']
df_2000['DP04_0002E'] = round(df_2000['DP04_0001E'] * df_2000['DP04_0002PE'].astype(float)/100)
df_2000['DP04_0017E'] = round(df_2000['DP04_0001E'] * df_2000['DP04_0017PE'].astype(float)/100)
df_2000['DP04_0018E'] = round(df_2000['DP04_0001E'] * df_2000['DP04_0018PE'].astype(float)/100)
df_2000['DP04_0019E'] = round(df_2000['DP04_0001E'] * df_2000['DP04_0019PE'].astype(float)/100)
df_2000['DP04_0020E'] = round(df_2000['DP04_0001E'] * df_2000['DP04_0020PE'].astype(float)/100)
df_2000['DP04_0021E'] = round(df_2000['DP04_0001E'] * df_2000['DP04_0021PE'].astype(float)/100)
df_2000['DP04_0022E'] = round(df_2000['DP04_0001E'] * df_2000['DP04_0022PE'].astype(float)/100)
df_2000['DP04_0023E'] = round(df_2000['DP04_0001E'] * df_2000['DP04_0023PE'].astype(float)/100)
df_2000['DP04_0024E'] = round(df_2000['DP04_0001E'] * df_2000['DP04_0024PE'].astype(float)/100)
df_2000['DP04_0025E'] = round(df_2000['DP04_0001E'] * df_2000['DP04_0025PE'].astype(float)/100)
df_2000['DP04_0026E'] = round(df_2000['DP04_0001E'] * df_2000['DP04_0026PE'].astype(float)/100)

df_2000 = df_2000.iloc[:,0:27]

# Adjust df_2000 dataframe columns to align with df_2023
df_2000['DP04_0026PE'] = df_2000['DP04_0025PE']
df_2000['DP04_0026E'] = df_2000['DP04_0025E']
df_2000['DP04_0025PE'] = df_2000['DP04_0024PE']
df_2000['DP04_0025E'] = df_2000['DP04_0024E']
df_2000['DP04_0024PE'] = df_2000['DP04_0023PE']
df_2000['DP04_0024E'] = df_2000['DP04_0023E']
df_2000['DP04_0023PE'] = df_2000['DP04_0022PE']
df_2000['DP04_0023E'] = df_2000['DP04_0022E']
df_2000['DP04_0022PE'] = df_2000['DP04_0021PE']
df_2000['DP04_0022E'] = df_2000['DP04_0021E']
df_2000['DP04_0021PE'] = df_2000['DP04_0020PE']
df_2000['DP04_0021E'] = df_2000['DP04_0020E']
df_2000['DP04_0020PE'] = pd.to_numeric(df_2000['DP04_0019PE'][1:], errors='ignore') + pd.to_numeric(df_2000['DP04_0018PE'][1:], errors='ignore') + pd.to_numeric(df_2000['DP04_0017PE'][1:], errors='ignore')
df_2000['DP04_0020E'] = pd.to_numeric(df_2000['DP04_0019E'][1:], errors='ignore') + pd.to_numeric(df_2000['DP04_0018E'][1:], errors='ignore') + pd.to_numeric(df_2000['DP04_0017E'][1:], errors='ignore')
df_2000['DP04_0019E'][1:] = 0
df_2000['DP04_0017E'][1:] = 0
df_2000['DP04_0018E'][1:] = 0
df_2000['DP04_0019PE'][1:] = 0
df_2000['DP04_0017PE'][1:] = 0
df_2000['DP04_0018PE'][1:] = 0

# Join the dataframe together for the captured years
census_df = pd.concat([df_2023, df_2019, df_2015, df_2010, df_2000], axis=0)

# Change column names
census_df.columns=census_df.iloc[0]
census_df = census_df.rename(columns={'Geography': 'FIPS', 2022: 'Year'})

# Retain only the desired states and territories
filtered_census_df = census_df[census_df['Geographic Area Name'].str.contains(states_string)]
#filtered_census_df = census_df[~census_df['FIPS'].str.contains('US54')]

# Change orginal geopraphy code to FIPS code
filtered_census_df['FIPS'] = [int(code.split('US')[1]) for code in filtered_census_df['FIPS']]

# Convert dataframe columns to appropriate types
for col in filtered_census_df.columns:
  filtered_census_df[col] = pd.to_numeric(filtered_census_df[col], errors='ignore')

# Filter out "West Virginia"
filtered_census_df = filtered_census_df[(filtered_census_df['FIPS'] < 54000) |
                                        (filtered_census_df['FIPS'] > 70000)]



filtered_census_df.shape
#filtered_census_df[filtered_census_df['Year'] == 2000].head()
#int(FIPS_state_dict['01'].replace(',',''))
#filtered_census_df.to_excel(r'/census_data.xlsx', index=False)


/var/folders/15/mh2mgzhn1191h4zq_5d03wfc0000gn/T/ipykernel_33462/3669881709.py:19: DtypeWarning: Columns (2,3,4,6,7,8,9,10,11,12,13,14,15,18,19,22,23,24,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,146,147,148,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,178,179,180,181,182,183,184,185,186,187,190,191,194,195,196,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,

(5322, 27)

In [4]:
### Interpolation and Forecasting

# Create list of unique FIPS codes
fips_codes = filtered_census_df['FIPS'].unique()

# Create new dataframe two rows with Nan values for the missing years
df_nan = pd.DataFrame(columns=filtered_census_df.columns)

# List of years to interpolate
missing_years = [2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2010,
                 2011, 2012, 2013, 2015, 2016, 2017, 2019, 2020, 2021]

for n in range(len(missing_years)):
  df_nan.loc[n] = np.nan
  df_nan['Year'][n] = missing_years[n]


# Initialize new dataframe
interpolated_df = pd.DataFrame(columns=filtered_census_df.columns)

# Loop to first interpolate then forecast values for each FIPS code

for code in fips_codes:
  temp_df = filtered_census_df[filtered_census_df['FIPS'] == code]
  geo_name = temp_df['Geographic Area Name'].iloc[0]
  temp_df = pd.concat([temp_df, df_nan], axis=0)
  temp_df = temp_df.sort_values(by='Year')
  temp_df.interpolate(method='linear',  limit_direction = 'both', axis=0, inplace=True)
  temp_df['Geographic Area Name'] = geo_name


  # Forecast 2023 and 2024 values
  for forecast_year in range(2023,2025):
    temp_df.reset_index(inplace=True, drop=True)
    temp_df.loc[len(temp_df)] = 2*temp_df.loc[len(temp_df)-1][2:] - temp_df.loc[len(temp_df)-2][2:]
    temp_df.loc[len(temp_df)-1, 'Year'] = forecast_year
    temp_df.loc[len(temp_df)-1, 'Geographic Area Name'] = geo_name
    temp_df.loc[len(temp_df)-1, 'FIPS'] = code

  interpolated_df = pd.concat([interpolated_df, temp_df], axis=0)


interpolated_df.sort_values(by=['FIPS','Year'], inplace=True)


/var/folders/15/mh2mgzhn1191h4zq_5d03wfc0000gn/T/ipykernel_33462/1818806836.py:15: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_nan['Year'][n] = missing_years[n]
/var/folders/15/mh2mgzhn1191h4zq_5d03wfc0000gn/T/ipykernel_33462/1818806836

In [5]:
# Reformat Final Dataframe

census_final_df = interpolated_df
census_final_df.iloc[:,2:5] = round(census_final_df.iloc[:,2:5].astype(int))

col_names = census_final_df.columns

for col in col_names[5:]:

  if 'Estimate' in col:
    census_final_df[col] = round(census_final_df[col].astype(int))

  elif 'Percent' in col:
    census_final_df[col] = round(census_final_df[col].astype(float),1)


census_final_df
census_final_df.to_csv(r'census_data.csv', index=False)